<a href="https://colab.research.google.com/github/MPrazzoli/corpsol/blob/main/Script_buono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install QuantLib

     |████████████████████████████████| 20.1MB 1.4MB/s 


In [5]:
import os
import math
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date
path = '/content/drive/MyDrive/data.xlsx'

***Inserimento degli Input***

In [6]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

Inserire il sottostante: 
Inserire il piano di ammortamento: 
Inserire il nozionale: 
Inserire il tasso annuo: 
Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): 
Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: 
Inserire il periodo che intercorre tra le rate in numero di mesi: 
Inserire la convenzione:
Inserire la regola per il calcolo delle date:


***Schedule Date***

In [7]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor = Period(tenor, Months)
endofmonth = False
terminationDateConvention = business_convention
df = pd.DataFrame({'startdate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[:-1],'enddate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[1:]})

In [8]:
# Calcolo numero delle rate totali dell'ammortamento
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor) * int(maturity_label.rsplit(maturity_label[-1], 1)[0]))

***Ammortamento Francese***

In [9]:
def ammortamento_francese(amortization_rate,tenor,rate_totali,notional):
  
  t = (1 + amortization_rate / (12 / tenor)) ** rate_totali
  rata_fra = round(notional * t * (amortization_rate / (12 / tenor)) / (t - 1),2)
  Residuo_fra = notional
  Estinto_fra = 0
  tot_I_fra = 0
  plan_fra = np.array
  plan_fra = [[0,0,0,0,0, Residuo_fra]]
  rate_fra = range(0, rate_totali)
  for count in rate_fra:
    Int_quota_fra = round(Residuo_fra * (amortization_rate*(tenor/12)),2)
    Cap_quota_fra = (rata_fra - Int_quota_fra)
    tot_I_fra = (tot_I_fra + Int_quota_fra)
    Residuo_fra = (Residuo_fra - Cap_quota_fra)
    Estinto_fra = (Estinto_fra + Cap_quota_fra)
    Tot_fra = (Int_quota_fra + Cap_quota_fra)
    plan.append([Int_quota_fra, Cap_quota_fra, Tot_fra, tot_I_fra, Estinto_fra, Residuo_fra])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_fra = pd.DataFrame(plan_fra)
  df_fra.columns = index
  df1_fra = pd.concat([df,df_fra], axis=1)
  df2_fra = pd.concat([df,df_fra['Debito Residuo']], axis=1)[:-1]

  return (df2_fra)

***Ammortamento Italiano***


In [10]:
def ammortamento_italiano(amortization_rate,tenor,rate_totali, notional):

  Residuo_ita = notional
  tot_I_ita = 0
  Estinto_ita = 0
  Tot_ita = 0
  plan_ita = np.array
  plan_ita = [[0,0,0,0,0,Residuo_ita]]
  rate_ita = range(0,rate_totali)
  for count in rate_ita:
    Int_quota_ita = round(Residuo_ita*amortization_rate/(12/tenor),2)
    Cap_quota_ita = round((notional/rate_totali),2)
    tot_I_ita = (tot_I_ita + Int_quota_ita)
    Residuo_ita = (Residuo_ita - Cap_quota_ita)
    Estinto_ita = (Estinto_ita + Cap_quota_ita)
    Tot_ita = (Int_quota_ita + Cap_quota_ita)
    plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_ita = pd.DataFrame(plan_ita)
  df_ita.columns=index
  df1_ita = pd.concat([df,df_ita],axis=1)
  df2_ita = pd.concat([df,df_ita['Debito Residuo']],axis=1)[:-1]
  
  return (df2_ita)
  

***Ammortamento Bullet***


In [11]:
def ammortamento_bullet(amortization_rate,tenor,rate_totali, notional):

  Residuo_bullet = notional
  tot_I_bullet = 0
  plan_bullet = np.array
  plan_bullet = [[0,0,Residuo_bullet]]
  rate_bullet = range(0,rate_totali)
  for count in rate_bullet:
    Int_quota_bullet = round(Residuo_bullet*amortization_rate/(12/tenor),2)
    tot_I_bullet = (tot_I_bullet+Int_quota_bullet)
    plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])
  index = ['Quota Interesse','Interessi Totali','Debito Residuo']
  df_bullet = pd.DataFrame(plan_bullet)
  df_bullet.columns = index
  df1_bullet = pd.concat([df,df_bullet],axis=1)
  df2_bullet = pd.concat([df,df_bullet['Debito Residuo']],axis=1)
  df2_bullet = df2_bullet[:-1]
  
  return (df2_bullet)

***Output del Modello***


In [12]:
if (amortization_type == 'Francese'):
  df_output = (ammortamento_francese(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Italiano'):
  df_output = (ammortamento_italiano(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Bullet'):
  df_output = (ammortamento_bullet(amortization_rate,tenor,rate_totali,notional))
  
print (df_output)

             startdate             enddate  Debito Residuo
0     April 30th, 2021     July 30th, 2021        100000.0
1      July 30th, 2021  October 29th, 2021        100000.0
2   October 29th, 2021  January 31st, 2022        100000.0
3   January 31st, 2022    April 29th, 2022        100000.0
4     April 29th, 2022     July 29th, 2022        100000.0
5      July 29th, 2022  October 31st, 2022        100000.0
6   October 31st, 2022  January 30th, 2023        100000.0
7   January 30th, 2023    April 28th, 2023        100000.0
8     April 28th, 2023     July 31st, 2023        100000.0
9      July 31st, 2023  October 30th, 2023        100000.0
10  October 30th, 2023  January 30th, 2024        100000.0
11  January 30th, 2024    April 30th, 2024        100000.0
12    April 30th, 2024     July 30th, 2024        100000.0
13     July 30th, 2024  October 30th, 2024        100000.0
14  October 30th, 2024  January 30th, 2025        100000.0
15  January 30th, 2025    April 30th, 2025        100000

In [77]:
fixing_date=[]
fixing_date_start=[]
for i, date in enumerate(df_output['startdate']):
  date_fix = (calendar.advance(date,-2, Days))
  fixing_date.append(date_fix)
  fixing_date_start.append(calendar.advance(date_fix, 2, Days))
df_output['fixing rate'] = fixing_date
df_output['start']= fixing_date_start
df_output['end'] = fixing_date_start[1:]






ValueError: ignored

In [67]:
fixing_date

[Date(28,4,2021),
 Date(28,7,2021),
 Date(27,10,2021),
 Date(27,1,2022),
 Date(27,4,2022),
 Date(27,7,2022),
 Date(27,10,2022),
 Date(26,1,2023),
 Date(26,4,2023),
 Date(27,7,2023),
 Date(26,10,2023),
 Date(26,1,2024),
 Date(26,4,2024),
 Date(26,7,2024),
 Date(28,10,2024),
 Date(28,1,2025),
 Date(28,4,2025),
 Date(28,7,2025),
 Date(28,10,2025),
 Date(28,1,2026)]

In [75]:
df_output

,startdate,enddate,Debito Residuo,fixing rate,start,end
0,"April 30th, 2021","July 30th, 2021",100000.0,"April 28th, 2021","April 30th, 2021","January 30th, 2026"
1,"July 30th, 2021","October 29th, 2021",100000.0,"July 28th, 2021","July 30th, 2021","January 30th, 2026"
2,"October 29th, 2021","January 31st, 2022",100000.0,"October 27th, 2021","October 29th, 2021","January 30th, 2026"
3,"January 31st, 2022","April 29th, 2022",100000.0,"January 27th, 2022","January 31st, 2022","January 30th, 2026"
4,"April 29th, 2022","July 29th, 2022",100000.0,"April 27th, 2022","April 29th, 2022","January 30th, 2026"
5,"July 29th, 2022","October 31st, 2022",100000.0,"July 27th, 2022","July 29th, 2022","January 30th, 2026"
6,"October 31st, 2022","January 30th, 2023",100000.0,"October 27th, 2022","October 31st, 2022","January 30th, 2026"
7,"January 30th, 2023","April 28th, 2023",100000.0,"January 26th, 2023","January 30th, 2023","January 30th, 2026"
8,"April 28th, 2023","July 31st, 2023",100000.0,"April 26th, 2023","April 28th, 2023","January 30th, 2026"
9,"July 31st, 2023","October 30th, 2023",100000.0,"July 27th, 2023","July 31st, 2023","January 30th, 2026"


Npv Obiettivo: prova

In [ ]:
df_output['differenza_giorni'] = (df_output['enddate'] - df_output['startdate']) / 360
df_output['prod'] = (df_output['Debito Residuo'] * df_output['differenza_giorni']) * 0.0001
BPV = df_output['prod'].sum()

*Class: Market Data*

In [ ]:
class historical_fixed_rate(object):

    def __init__(self, name, reference_date = None, currency = None, underlying = None):
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
        
        self.name = name
        self.currency = currency
        self.underlying = underlying
        self.historical_fixed_rate_df = pd.DataFrame

class zero_curve_rate(object):
# chiedere a Mirko l'assegnazione di interp_rule e day_count
    def __init__(self, name, reference_date = None, currency = None, underlying = None, interp_rule = None, day_count = None):
        
        self.name = name
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
      
        self.currency = currency
        self.underlying = underlying
        self.interpolation_rule = interp_rule
        self.day_count = day_count
        self.zero_curve_rate_df = pd.DataFrame

In [ ]:
def market_data(path):

  historical_fixed_rates = historical_fixed_rate(name="HISTORICAL RATE")
  historical_fixed_rates.historical_fixed_rate_df = pd.read_excel(path, sheet_name='fixings')

  estimation_curve = zero_curve_rate(name='ESTIMATION')
  estimation_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='estimation')

  discount_curve = zero_curve_rate(name='DISCOUNT')
  discount_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='discount')

  market_data_dict = {'ESTIMATION': estimation_curve, 'DISCOUNT': discount_curve, 'HISTORICAL': historical_fixed_rates}

  return(market_data_dict)

In [ ]:
m=market_data(path)
a=m['ESTIMATION'].zero_curve_rate_df['DATE']


In [ ]:
class IRS:

  def __init__(self):

    self.start_date = None
    self.maturity_date = None
    self.tenor = None
    self.business_convention = None
    self.date_gen_rule = None
    self.schedule = pd.DataFrame
    self.eom = None

    
    self.npv = None

In [ ]:
def pricer(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

    dictionary = market_data(path)
    irs = IRS()
    irs.start_date = calendar.advance(ref_date,2,Days)
    if maturity_label[-1] == 'M':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
    elif maturity_label[-1] == 'Y':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
    irs.business_convention = business_convention
    irs.tenor = tenor
    irs.date_gen_rule = date_gen_rule
    irs.eom = end_of_month

# aggiungere altri attribuiti all'irs(calendario,EOM, deduction formula, day count)


    
    irs.schedule = pd.DataFrame({'startdate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[:-1],'enddate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[1:]})

In [ ]:
class IRS:

  def __init__(self, leg1_name = 'Receive', leg2_name = 'Pay', leg1_rate_type = 'Fixed', leg2_rate_type = 'Floating'):

    self.start_date = None
    self.leg1_flows = pd.DataFrame
    self.leg2_flows = pd.DataFrame
    self.leg1_name = leg1_name
    self.leg2_name = leg2_name
    self.leg1_rate_type = leg1_rate_type  
    self.leg2_rate_type = leg2_rate_type
    self.npv = None

In [ ]:
def submit(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

  irs = IRS()

In [ ]:
submit(path = market_data_path, underlying = underlying, amortization_type = amortization_type, notional = notional, amortization_rate = amortization_rate, maturity_label = maturity_label, ref_date = ref_date, tenor = tenor,
       business_convention = business_convention, date_gen_rule = date_gen_rule)

In [ ]:
leg_fixed=FixedRateLeg(schedule,Actual360(),[notional],[0.05])
for i in leg_fixed:
  print(i.date(),i.amount())